In [1]:
!nvidia-smi

Wed May  7 08:12:17 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.124.06             Driver Version: 570.124.06     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA L4                      Off |   00000000:00:03.0 Off |                    0 |
| N/A   41C    P8             16W /   72W |       1MiB /  23034MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [2]:
import torch, torchvision, torchaudio

In [3]:
torch.cuda.is_available()

True

In [4]:
torch.cuda.device_count()

1

In [5]:
torch.cuda.current_device()

0

In [6]:
torch.cuda.device(0)

In [7]:
torch.cuda.get_device_name(0)

'NVIDIA L4'

# Data

In [8]:
!pip install git+https://github.com/Atze00/MoViNet-pytorch.git

  Cloning https://github.com/Atze00/MoViNet-pytorch.git to /tmp/pip-req-build-ne93tmyv
  Running command git clone --filter=blob:none --quiet https://github.com/Atze00/MoViNet-pytorch.git /tmp/pip-req-build-ne93tmyv
  Resolved https://github.com/Atze00/MoViNet-pytorch.git to commit c2d1edf48fc6c5259707f9d833f22171b4f63493
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for MoViNet-pytorch: filename=movinet_pytorch-0.2.2-py3-none-any.whl size=12037 sha256=7664eed62d0ff7f20abc37290173904797e6e8692aacd0fa981f067fc789e9be
  Stored in directory: /tmp/pip-ephem-wheel-cache-qzvsj4i5/wheels/dd/e3/c8/08203f421d88d2ff3efaeb2384d3

In [9]:
import time
import torchvision
import torch.nn.functional as F
import torchvision.transforms as T
import torch.optim as optim
from torch.utils.data import random_split, DataLoader
import torch
# import transforms as T
from movinets import MoViNet
from movinets.config import _C

In [10]:
import sys
sys.path

['/home/gabir/workspace/repos/ai-system-architecture',
 '/opt/conda/lib/python310.zip',
 '/opt/conda/lib/python3.10',
 '/opt/conda/lib/python3.10/lib-dynload',
 '',
 '/opt/conda/lib/python3.10/site-packages']

In [11]:
%pwd

'/home/gabir/workspace/repos/ai-system-architecture'

In [12]:
#
# Load Xs and ys from pickle
import pickle

with open("Xs_final.pickle", "rb") as read_file:
  X_new = pickle.load(read_file)

with open("ys_final.pickle", "rb") as read_file:
  y_new = pickle.load(read_file)

In [13]:
len(X_new)

1241

In [14]:
len(y_new)

1241

In [15]:
with open("Xs_final_test.pickle", "rb") as read_file:
  X_new_test = pickle.load(read_file)

with open("ys_final_test.pickle", "rb") as read_file:
  y_new_test = pickle.load(read_file)

In [16]:
len(X_new_test)

932

In [17]:
len(y_new_test)

932

In [18]:
y_set = set(y_new)

class_counts = {k: 0 for k in list(y_set)}

for label in y_new:
  class_counts[label] += 1

class_counts

{0: 250, 1: 178, 2: 213, 3: 377, 4: 223}

In [19]:
#
# Calculate class weights
class_weights = {}
i = 0
for k, v in class_counts.items():
  class_weights[i] = len(X_new) / (len(class_counts)*v)
  i += 1
class_weights

{0: 0.9928,
 1: 1.3943820224719101,
 2: 1.1652582159624414,
 3: 0.6583554376657825,
 4: 1.1130044843049327}

In [20]:
import numpy as np

Prepare paths a little instead of dividing into train and test as it is already done

In [21]:
X_new

['train_data/kicks_01_segment_0.avi',
 'train_data/kicks_01_segment_1.avi',
 'train_data/kicks_01_segment_5.avi',
 'train_data/kicks_01_segment_6.avi',
 'train_data/kicks_01_segment_7.avi',
 'train_data/kicks_01_segment_8.avi',
 'train_data/kicks_01_segment_11.avi',
 'train_data/kicks_01_segment_14.avi',
 'train_data/kicks_01_segment_21.avi',
 'train_data/kicks_01_segment_22.avi',
 'train_data/kicks_01_segment_23.avi',
 'train_data/kicks_01_segment_26.avi',
 'train_data/kicks_01_segment_29.avi',
 'train_data/kicks_01_segment_30.avi',
 'train_data/kicks_01_segment_31.avi',
 'train_data/kicks_01_segment_32.avi',
 'train_data/kicks_01_segment_36.avi',
 'train_data/kicks_01_segment_37.avi',
 'train_data/kicks_01_segment_38.avi',
 'train_data/kicks_01_segment_39.avi',
 'train_data/kicks_01_segment_47.avi',
 'train_data/kicks_01_segment_48.avi',
 'train_data/kicks_01_segment_49.avi',
 'train_data/kicks_01_segment_52.avi',
 'train_data/kicks_01_segment_53.avi',
 'train_data/kicks_01_segment_5

In [22]:
for i in range(len(X_new)):
    X_new[i] = X_new[i][:19] + "/" + X_new[i][20:]

X_new

['train_data/kicks_01/segment_0.avi',
 'train_data/kicks_01/segment_1.avi',
 'train_data/kicks_01/segment_5.avi',
 'train_data/kicks_01/segment_6.avi',
 'train_data/kicks_01/segment_7.avi',
 'train_data/kicks_01/segment_8.avi',
 'train_data/kicks_01/segment_11.avi',
 'train_data/kicks_01/segment_14.avi',
 'train_data/kicks_01/segment_21.avi',
 'train_data/kicks_01/segment_22.avi',
 'train_data/kicks_01/segment_23.avi',
 'train_data/kicks_01/segment_26.avi',
 'train_data/kicks_01/segment_29.avi',
 'train_data/kicks_01/segment_30.avi',
 'train_data/kicks_01/segment_31.avi',
 'train_data/kicks_01/segment_32.avi',
 'train_data/kicks_01/segment_36.avi',
 'train_data/kicks_01/segment_37.avi',
 'train_data/kicks_01/segment_38.avi',
 'train_data/kicks_01/segment_39.avi',
 'train_data/kicks_01/segment_47.avi',
 'train_data/kicks_01/segment_48.avi',
 'train_data/kicks_01/segment_49.avi',
 'train_data/kicks_01/segment_52.avi',
 'train_data/kicks_01/segment_53.avi',
 'train_data/kicks_01/segment_5

In [23]:
len(X_new)

1241

In [24]:
import os

In [25]:
for i in range(len(X_new)):
    path_splits = X_new[i].split("/")
    X_new[i] = os.path.join("final_dataset", *path_splits[1:])

X_new

['final_dataset/kicks_01/segment_0.avi',
 'final_dataset/kicks_01/segment_1.avi',
 'final_dataset/kicks_01/segment_5.avi',
 'final_dataset/kicks_01/segment_6.avi',
 'final_dataset/kicks_01/segment_7.avi',
 'final_dataset/kicks_01/segment_8.avi',
 'final_dataset/kicks_01/segment_11.avi',
 'final_dataset/kicks_01/segment_14.avi',
 'final_dataset/kicks_01/segment_21.avi',
 'final_dataset/kicks_01/segment_22.avi',
 'final_dataset/kicks_01/segment_23.avi',
 'final_dataset/kicks_01/segment_26.avi',
 'final_dataset/kicks_01/segment_29.avi',
 'final_dataset/kicks_01/segment_30.avi',
 'final_dataset/kicks_01/segment_31.avi',
 'final_dataset/kicks_01/segment_32.avi',
 'final_dataset/kicks_01/segment_36.avi',
 'final_dataset/kicks_01/segment_37.avi',
 'final_dataset/kicks_01/segment_38.avi',
 'final_dataset/kicks_01/segment_39.avi',
 'final_dataset/kicks_01/segment_47.avi',
 'final_dataset/kicks_01/segment_48.avi',
 'final_dataset/kicks_01/segment_49.avi',
 'final_dataset/kicks_01/segment_52.avi'

In [26]:
len(X_new)

1241

In [28]:
for i in range(len(X_new)):
    path_splits = X_new[i].split("/")
    X_new[i] = os.path.join(*path_splits[:2], "segmented", *path_splits[2:])

X_new

['final_dataset/kicks_01/segmented/segment_0.avi',
 'final_dataset/kicks_01/segmented/segment_1.avi',
 'final_dataset/kicks_01/segmented/segment_5.avi',
 'final_dataset/kicks_01/segmented/segment_6.avi',
 'final_dataset/kicks_01/segmented/segment_7.avi',
 'final_dataset/kicks_01/segmented/segment_8.avi',
 'final_dataset/kicks_01/segmented/segment_11.avi',
 'final_dataset/kicks_01/segmented/segment_14.avi',
 'final_dataset/kicks_01/segmented/segment_21.avi',
 'final_dataset/kicks_01/segmented/segment_22.avi',
 'final_dataset/kicks_01/segmented/segment_23.avi',
 'final_dataset/kicks_01/segmented/segment_26.avi',
 'final_dataset/kicks_01/segmented/segment_29.avi',
 'final_dataset/kicks_01/segmented/segment_30.avi',
 'final_dataset/kicks_01/segmented/segment_31.avi',
 'final_dataset/kicks_01/segmented/segment_32.avi',
 'final_dataset/kicks_01/segmented/segment_36.avi',
 'final_dataset/kicks_01/segmented/segment_37.avi',
 'final_dataset/kicks_01/segmented/segment_38.avi',
 'final_dataset/ki

In [29]:
len(X_new)

1241

Same for test dataset

In [30]:
for i in range(len(X_new_test)):
    X_new_test[i] = X_new_test[i][:18] + "/" + X_new_test[i][19:]

X_new_test

['test_data/kicks_27/segment_0.avi',
 'test_data/kicks_27/segment_1.avi',
 'test_data/kicks_27/segment_2.avi',
 'test_data/kicks_27/segment_3.avi',
 'test_data/kicks_27/segment_4.avi',
 'test_data/kicks_27/segment_5.avi',
 'test_data/kicks_27/segment_6.avi',
 'test_data/kicks_27/segment_7.avi',
 'test_data/kicks_27/segment_8.avi',
 'test_data/kicks_27/segment_9.avi',
 'test_data/kicks_27/segment_10.avi',
 'test_data/kicks_27/segment_11.avi',
 'test_data/kicks_27/segment_12.avi',
 'test_data/kicks_27/segment_13.avi',
 'test_data/kicks_27/segment_14.avi',
 'test_data/kicks_27/segment_15.avi',
 'test_data/kicks_27/segment_16.avi',
 'test_data/kicks_27/segment_17.avi',
 'test_data/kicks_27/segment_18.avi',
 'test_data/kicks_27/segment_19.avi',
 'test_data/kicks_27/segment_20.avi',
 'test_data/kicks_27/segment_21.avi',
 'test_data/kicks_27/segment_22.avi',
 'test_data/kicks_27/segment_23.avi',
 'test_data/kicks_27/segment_24.avi',
 'test_data/kicks_27/segment_25.avi',
 'test_data/kicks_27/s

In [31]:
for i in range(len(X_new_test)):
    path_splits = X_new_test[i].split("/")
    X_new_test[i] = os.path.join("final_dataset", *path_splits[1:])

X_new_test

['final_dataset/kicks_27/segment_0.avi',
 'final_dataset/kicks_27/segment_1.avi',
 'final_dataset/kicks_27/segment_2.avi',
 'final_dataset/kicks_27/segment_3.avi',
 'final_dataset/kicks_27/segment_4.avi',
 'final_dataset/kicks_27/segment_5.avi',
 'final_dataset/kicks_27/segment_6.avi',
 'final_dataset/kicks_27/segment_7.avi',
 'final_dataset/kicks_27/segment_8.avi',
 'final_dataset/kicks_27/segment_9.avi',
 'final_dataset/kicks_27/segment_10.avi',
 'final_dataset/kicks_27/segment_11.avi',
 'final_dataset/kicks_27/segment_12.avi',
 'final_dataset/kicks_27/segment_13.avi',
 'final_dataset/kicks_27/segment_14.avi',
 'final_dataset/kicks_27/segment_15.avi',
 'final_dataset/kicks_27/segment_16.avi',
 'final_dataset/kicks_27/segment_17.avi',
 'final_dataset/kicks_27/segment_18.avi',
 'final_dataset/kicks_27/segment_19.avi',
 'final_dataset/kicks_27/segment_20.avi',
 'final_dataset/kicks_27/segment_21.avi',
 'final_dataset/kicks_27/segment_22.avi',
 'final_dataset/kicks_27/segment_23.avi',
 '

In [32]:
len(X_new_test)

932

In [33]:
for i in range(len(X_new_test)):
    path_splits = X_new_test[i].split("/")
    X_new_test[i] = os.path.join(*path_splits[:2], "segmented", *path_splits[2:])

X_new_test

['final_dataset/kicks_27/segmented/segment_0.avi',
 'final_dataset/kicks_27/segmented/segment_1.avi',
 'final_dataset/kicks_27/segmented/segment_2.avi',
 'final_dataset/kicks_27/segmented/segment_3.avi',
 'final_dataset/kicks_27/segmented/segment_4.avi',
 'final_dataset/kicks_27/segmented/segment_5.avi',
 'final_dataset/kicks_27/segmented/segment_6.avi',
 'final_dataset/kicks_27/segmented/segment_7.avi',
 'final_dataset/kicks_27/segmented/segment_8.avi',
 'final_dataset/kicks_27/segmented/segment_9.avi',
 'final_dataset/kicks_27/segmented/segment_10.avi',
 'final_dataset/kicks_27/segmented/segment_11.avi',
 'final_dataset/kicks_27/segmented/segment_12.avi',
 'final_dataset/kicks_27/segmented/segment_13.avi',
 'final_dataset/kicks_27/segmented/segment_14.avi',
 'final_dataset/kicks_27/segmented/segment_15.avi',
 'final_dataset/kicks_27/segmented/segment_16.avi',
 'final_dataset/kicks_27/segmented/segment_17.avi',
 'final_dataset/kicks_27/segmented/segment_18.avi',
 'final_dataset/kicks_

In [34]:
len(X_new_test)

932

In [18]:
#
# Div into train and test
val_indices = np.random.choice(np.arange(len(X_new)), size=100, replace=False)
val_X = [X_new[i] for i in val_indices]
train_X = [X_new[i] for i in range(len(X_new)) if i not in val_indices]

print(len(train_X))
print(len(val_X))

1803
100


In [19]:
#
val_y = [y_new[i] for i in val_indices]
train_y = [y_new[i] for i in range(len(y_new)) if i not in val_indices]

print(len(train_y))
print(len(val_y))

1803
100


In [35]:
import cv2
from tqdm import tqdm

In [36]:
# import torch
from torch.utils.data import Dataset, DataLoader

class KickPunchDataset(Dataset):
    def __init__(self, data, labels, n_frames, transform=None):
        self.data = data
        self.labels = labels
        self.n_frames = n_frames
        self.transform = transform
    
    def __len__(self):
        return len(self.data)
    
    def format_frames(self, frame):
        if self.transform:
            frame = self.transform(frame)
        return frame
    
    def load_video(self, video_path):
        # Read each video frame by frame
        result = []
        # print(video_path)
        src = cv2.VideoCapture(str(video_path))
        
        # video_length = src.get(cv2.CAP_PROP_FRAME_COUNT)
        
        # need_length = 1 + (n_frames - 1) * frame_step
        
        # if need_length > video_length:
        #   start = 0
        # else:
        #   max_start = video_length - need_length
        #   start = random.randint(0, max_start + 1)
        
        # src.set(cv2.CAP_PROP_POS_FRAMES, start)
        # ret is a boolean indicating whether read was successful, frame is the image itself
        # ret, frame = src.read()
        # result.append(format_frames(frame, output_size))
        
        for _ in range(self.n_frames):
            ret, frame = src.read()
            if ret:
                frame = self.format_frames(frame)#, output_size)
                result.append(frame)
            else:
                result.append(np.zeros_like(result[0]))
        src.release()
        result = np.array(result)#[..., [2, 1, 0]]
        
        return result
    
    def __getitem__(self, idx):
        res = self.load_video(self.data[idx])
        # label = torch.nn.functional.one_hot(self.labels[idx], 5)
        sample = {'data': res, 'label': self.labels[idx]}
        return sample

transform = T.Compose([  
                                 T.ToTensor(),                    
                                 T.Lambda(lambda x: x / 255.), # scale in [0, 1],
                                 T.Resize((200, 200)),
                                 T.RandomHorizontalFlip(),
                                 #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.RandomCrop((172, 172))])
transform_test = T.Compose([
    T.ToTensor(),
    T.Lambda(lambda x: x / 255.), # scale in [0, 1],
    T.Resize((200, 200)),
    #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
    T.CenterCrop((172, 172))])
    
# Example usage
#data = torch.randn(100, 3, 32, 32)  # Example data
#labels = torch.randint(0, 10, (100,))  # Example labels
# train_dataset = KickPunchDataset(train_X, train_y, 30, transform=transform) # old
# test_dataset = KickPunchDataset(val_X, val_y, 30, transform=transform_test) # old
train_dataset = KickPunchDataset(X_new, y_new, 30, transform=transform) # old
test_dataset = KickPunchDataset(X_new_test, y_new_test, 30, transform=transform_test) # old

# Create a DataLoader
batch_size = 4
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# Iterate over batches
#for batch in train_dataloader:
#    # Access batch['data'] and batch['label'] for each batch
#    print(batch['data'].shape, batch['label'].shape)
#    break


In [38]:
# NEED TO DEFINE OWN DATA LOADERS

torch.manual_seed(97)
num_frames = 16 # 16
clip_steps = 2
Bs_Train = 16
Bs_Test = 16

transform = transforms.Compose([  
                                 T.ToTensor(),                    
                                 T.ToFloatTensorInZeroOne(),
                                 T.Resize((200, 200)),
                                 T.RandomHorizontalFlip(),
                                 #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
                                 T.RandomCrop((172, 172))])
transform_test = transforms.Compose([
    T.ToTensor(),
    T.ToFloatTensorInZeroOne(),
    T.Resize((200, 200)),
    #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
    T.CenterCrop((172, 172))])

hmdb51_train = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,frame_rate=5,
                                                step_between_clips = clip_steps, fold=1, train=True,
                                                transform=transform, num_workers=2)


hmdb51_test = torchvision.datasets.HMDB51('video_data/', 'test_train_splits/', num_frames,frame_rate=5,
                                                step_between_clips = clip_steps, fold=1, train=False,
                                                transform=transform_test, num_workers=2)
train_loader = DataLoader(hmdb51_train, batch_size=Bs_Train, shuffle=True)
test_loader  = DataLoader(hmdb51_test, batch_size=Bs_Test, shuffle=False)

NameError: name 'transforms' is not defined

In [22]:
# Load the model

model = MoViNet(_C.MODEL.MoViNetA0, causal = False, pretrained = True )

In [23]:
model

MoViNet(
  (conv1): ConvBlock3D(
    (conv_1): Conv3DBNActivation(
      (conv3d): Conv3d(3, 8, kernel_size=(1, 3, 3), stride=(1, 2, 2), bias=False)
      (norm): BatchNorm3d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): Swish()
    )
  )
  (blocks): Sequential(
    (b0_l0): BasicBneck(
      (expand): ConvBlock3D(
        (conv_1): Conv3DBNActivation(
          (conv3d): Conv3d(8, 24, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (norm): BatchNorm3d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act): Swish()
        )
      )
      (deep): ConvBlock3D(
        (conv_1): Conv3DBNActivation(
          (conv3d): Conv3d(24, 24, kernel_size=(1, 5, 5), stride=(1, 2, 2), groups=24, bias=False)
          (norm): BatchNorm3d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act): Swish()
        )
      )
      (se): SqueezeExcitation(
        (temporal_cumualtive_GAvg3D): TemporalC

In [24]:
# Custom classifier

model.classifier[3] = torch.nn.Conv3d(2048, 5, (1,1,1))

In [25]:
model

MoViNet(
  (conv1): ConvBlock3D(
    (conv_1): Conv3DBNActivation(
      (conv3d): Conv3d(3, 8, kernel_size=(1, 3, 3), stride=(1, 2, 2), bias=False)
      (norm): BatchNorm3d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): Swish()
    )
  )
  (blocks): Sequential(
    (b0_l0): BasicBneck(
      (expand): ConvBlock3D(
        (conv_1): Conv3DBNActivation(
          (conv3d): Conv3d(8, 24, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (norm): BatchNorm3d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act): Swish()
        )
      )
      (deep): ConvBlock3D(
        (conv_1): Conv3DBNActivation(
          (conv3d): Conv3d(24, 24, kernel_size=(1, 5, 5), stride=(1, 2, 2), groups=24, bias=False)
          (norm): BatchNorm3d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act): Swish()
        )
      )
      (se): SqueezeExcitation(
        (temporal_cumualtive_GAvg3D): TemporalC

In [26]:
# Freezing all layers except classifier

for name, param in model.named_parameters():
    if not name in ["classifier.3.weight", "classifier.3.bias"]:
        param.requires_grad = False

In [26]:
for name, param in model.named_parameters():
    print(f'Layer: {name}, Requires Grad: {param.requires_grad}')

Layer: conv1.conv_1.conv3d.weight, Requires Grad: True
Layer: conv1.conv_1.norm.weight, Requires Grad: True
Layer: conv1.conv_1.norm.bias, Requires Grad: True
Layer: blocks.b0_l0.alpha, Requires Grad: True
Layer: blocks.b0_l0.expand.conv_1.conv3d.weight, Requires Grad: True
Layer: blocks.b0_l0.expand.conv_1.norm.weight, Requires Grad: True
Layer: blocks.b0_l0.expand.conv_1.norm.bias, Requires Grad: True
Layer: blocks.b0_l0.deep.conv_1.conv3d.weight, Requires Grad: True
Layer: blocks.b0_l0.deep.conv_1.norm.weight, Requires Grad: True
Layer: blocks.b0_l0.deep.conv_1.norm.bias, Requires Grad: True
Layer: blocks.b0_l0.se.fc1.conv_1.conv3d.weight, Requires Grad: True
Layer: blocks.b0_l0.se.fc1.conv_1.conv3d.bias, Requires Grad: True
Layer: blocks.b0_l0.se.fc2.conv_1.conv3d.weight, Requires Grad: True
Layer: blocks.b0_l0.se.fc2.conv_1.conv3d.bias, Requires Grad: True
Layer: blocks.b0_l0.project.conv_1.conv3d.weight, Requires Grad: True
Layer: blocks.b0_l0.project.conv_1.norm.weight, Requires

In [37]:
# Training functions

def calc_acc(y_true, y_pred):
    n_correct = torch.sum(y_true == y_pred)
    return n_correct/y_true.size(0)

def train_iter(model, optimz, loss_fn, data_load, loss_val, epoch):
    print("------------------------")
    print(f"EPOCH {epoch}")
    epoch_loss, epoch_acc = 0, 0
    samples = len(data_load.dataset)
    model.train()
    model.cuda()
    model.clean_activation_buffers()
    optimz.zero_grad()
    pbar = tqdm(enumerate(data_load))
    for i, sample in pbar:
        res = sample["data"].permute(0, 2, 1, 3, 4)
#         out = F.log_softmax(model(res.cuda()), dim=1)
#         loss = F.nll_loss(out, sample["label"].cuda())
        out = F.softmax(model(res.cuda()), dim=1)
        loss = loss_fn(out, sample["label"].cuda())
        loss.backward()
        optimz.step()
        optimz.zero_grad()
        model.clean_activation_buffers()
        _, pred = torch.max(out, dim=1)
        epoch_loss += loss.item()
        accuracy = calc_acc(sample["label"].cuda(), pred.cuda())
        epoch_acc += accuracy.item()
        # print(accuracy)
        pbar.set_description(f"{i+1} of {samples//batch_size + 1} batches passed. Loss: {round(epoch_loss / (i + 1), 5)}, Accuracy: {round(epoch_acc / (i + 1), 5)}")
        if i % 50 == 0:
        #     print('[' +  '{:5}'.format(i * len(data)) + '/' + '{:5}'.format(samples) + ' (' + '{:3.0f}'.format(100 * i / len(data_load)) + '%)]  Loss: ' + '{:6.4f}'.format(loss.item()))
            loss_val.append(loss.item())
 
def evaluate(model, loss_fn, data_load, loss_val):
    print("------------------------")
    model.eval()
    
    samples = len(data_load.dataset)
    csamp = 0
    tloss = 0
    model.clean_activation_buffers()
    with torch.no_grad():
        pbar = tqdm(enumerate(data_load))
        for i, sample in pbar:
            res = sample["data"].permute(0, 2, 1, 3, 4)
#             output = F.log_softmax(model(res.cuda()), dim=1)
#             loss = F.nll_loss(output, sample["label"].cuda(), reduction='sum')
            output = F.softmax(model(res.cuda()), dim=1)
            loss = loss_fn(output, sample["label"].cuda())
            _, pred = torch.max(output, dim=1)
            
            tloss += loss.item()
            csamp += pred.eq(sample["label"].cuda()).sum()
            model.clean_activation_buffers()
            pbar.set_description(f"{i+1} of {samples//batch_size + 1} batches passed. Loss: {round(tloss / (i + 1), 5)}, Accuracy: {100.0 * csamp.item() / (i + 1), 5}")
    aloss = tloss / samples
    loss_val.append(aloss)
    print('\nAverage test loss: ' + '{:.4f}'.format(aloss) + '  Accuracy:' + '{:5}'.format(csamp) + '/' + '{:5}'.format(samples) + ' (' + '{:4.2f}'.format(100.0 * csamp / samples) + '%)\n')

In [38]:
weight_tensor = torch.Tensor(list(class_weights.values()))
weight_tensor

tensor([0.9928, 1.3944, 1.1653, 0.6584, 1.1130])

In [39]:
model = torch.load("weights/movinet_ft_10_epoch_val_acc_43.pth")

In [40]:
model

MoViNet(
  (conv1): ConvBlock3D(
    (conv_1): Conv3DBNActivation(
      (conv3d): Conv3d(3, 8, kernel_size=(1, 3, 3), stride=(1, 2, 2), bias=False)
      (norm): BatchNorm3d(8, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
      (act): Swish()
    )
  )
  (blocks): Sequential(
    (b0_l0): BasicBneck(
      (expand): ConvBlock3D(
        (conv_1): Conv3DBNActivation(
          (conv3d): Conv3d(8, 24, kernel_size=(1, 1, 1), stride=(1, 1, 1), bias=False)
          (norm): BatchNorm3d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act): Swish()
        )
      )
      (deep): ConvBlock3D(
        (conv_1): Conv3DBNActivation(
          (conv3d): Conv3d(24, 24, kernel_size=(1, 5, 5), stride=(1, 2, 2), groups=24, bias=False)
          (norm): BatchNorm3d(24, eps=0.001, momentum=0.1, affine=True, track_running_stats=True)
          (act): Swish()
        )
      )
      (se): SqueezeExcitation(
        (temporal_cumualtive_GAvg3D): TemporalC

In [41]:
# Training loop

N_EPOCHS = 30

# model = MoViNet(_C.MODEL.MoViNetA0, causal = False, pretrained = True )
start_time = time.time()

trloss_val, tsloss_val = [], []
# model.classifier[3] = torch.nn.Conv3d(2048, 51, (1,1,1))
optimz = optim.Adam(model.parameters(), lr=0.00005)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer=optimz, T_max=10, eta_min=0.000005)
train_loss = torch.nn.CrossEntropyLoss(weight=weight_tensor, label_smoothing=0.01)
test_loss = torch.nn.CrossEntropyLoss()
for epoch in range(1, N_EPOCHS + 1):
    # print('Epoch:', epoch)
    train_iter(model, scheduler, train_loss, train_dataloader, trloss_val, epoch)
    evaluate(model, test_loss, test_dataloader, tsloss_val)
 
print('Execution time:', '{:5.2f}'.format(time.time() - start_time), 'seconds')

------------------------
EPOCH 1


0it [00:00, ?it/s]/opt/conda/lib/python3.10/site-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(
111 of 311 batches passed. Loss: 1.35475, Accuracy: 0.0045: : 111it [02:24,  1.30s/it] 


KeyboardInterrupt: 

In [30]:
torch.save(model, "weights/movinet_ft_10_epoch_val_acc_43.pth")

In [ ]:
################# INFERENCE

In [2]:
model = torch.load("weights/movinet_ft_10_epoch_val_acc_43.pth")

In [3]:
!ls

Dockerfile			 entry.py		   poetry.lock
LICENSE.txt			 final_dataset		   pyproject.toml
MANIFEST.in			 final_dataset_2.zip	   requirements.txt
Makefile			 infra			   runs
README.md			 jupyter-venv		   scripts
Xs.pickle			 main.py		   setup.cfg
__MACOSX			 mamonbest947oscombo.hdfs  setup.py
_design				 mma			   test.jpg
bitbucket-pipelines.yml		 movinet_torch.ipynb	   test1.jpg
cnn_lstm.ipynb			 new_Xs.pickle		   test12.jpg
cuda_11.7.1_515.65.01_linux.run  new_ys.pickle		   tests
data				 nohup.out		   weights
datasets			 output_video.mp4	   ys.pickle
docker_setup.sh			 outputs


In [4]:
%pwd

'/home/gabir/workspace/repos/ai-system-architecture'

In [5]:
%cd final_dataset

/home/gabir/workspace/repos/ai-system-architecture/final_dataset


In [6]:
%pwd

'/home/gabir/workspace/repos/ai-system-architecture/final_dataset'

In [7]:
%ls

kicks_01/  kicks_06/  kicks_11/  kicks_16/  kicks_21/  kicks_26/  kicks_31/
kicks_02/  kicks_07/  kicks_12/  kicks_17/  kicks_22/  kicks_27/  kicks_32/
kicks_03/  kicks_08/  kicks_13/  kicks_18/  kicks_23/  kicks_28/  kicks_33/
kicks_04/  kicks_09/  kicks_14/  kicks_19/  kicks_24/  kicks_29/
kicks_05/  kicks_10/  kicks_15/  kicks_20/  kicks_25/  kicks_30/


In [8]:
%cd kicks_01

/home/gabir/workspace/repos/ai-system-architecture/final_dataset/kicks_01


In [9]:
%ls

instances_default.json  kicks_01.mp4  segmented/


In [10]:
%cd segmented

/home/gabir/workspace/repos/ai-system-architecture/final_dataset/kicks_01/segmented


In [11]:
%ls

segment_0.avi   segment_25.avi  segment_41.avi  segment_58.avi  segment_74.avi
segment_1.avi   segment_26.avi  segment_42.avi  segment_59.avi  segment_75.avi
segment_10.avi  segment_27.avi  segment_43.avi  segment_6.avi   segment_76.avi
segment_11.avi  segment_28.avi  segment_44.avi  segment_60.avi  segment_77.avi
segment_12.avi  segment_29.avi  segment_45.avi  segment_61.avi  segment_78.avi
segment_13.avi  segment_3.avi   segment_46.avi  segment_62.avi  segment_79.avi
segment_14.avi  segment_30.avi  segment_47.avi  segment_63.avi  segment_8.avi
segment_15.avi  segment_31.avi  segment_48.avi  segment_64.avi  segment_80.avi
segment_16.avi  segment_32.avi  segment_49.avi  segment_65.avi  segment_81.avi
segment_17.avi  segment_33.avi  segment_5.avi   segment_66.avi  segment_82.avi
segment_18.avi  segment_34.avi  segment_50.avi  segment_67.avi  segment_83.avi
segment_19.avi  segment_35.avi  segment_51.avi  segment_68.avi  segment_84.avi
segment_2.avi   segment_36.avi  segment_52.avi  segme

In [23]:
%cd ..

/home/gabir/workspace/repos/ai-system-architecture


In [33]:
import cv2
import numpy as np
# Read each video frame by frame

test_transform = T.Compose([
    T.ToTensor(),
    T.Lambda(lambda x: x / 255.), # scale in [0, 1],
    T.Resize((200, 200)),
    #T.Normalize(mean=[0.43216, 0.394666, 0.37645], std=[0.22803, 0.22145, 0.216989]),
    T.CenterCrop((172, 172))])

def transform_frames(frame):
    frame = test_transform(frame)
    return frame

result = []
src = cv2.VideoCapture("final_dataset/kicks_01/segmented/segment_50.avi")

for i in range(30):
    ret, frame = src.read()
    print(i)
    if ret:
        frame = transform_frames(frame)#, output_size)
        result.append(frame)
    else:
        result.append(np.zeros_like(result[0]))
src.release()
result = np.array(result)#[..., [2, 1, 0]]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29


In [34]:
result = torch.from_numpy(result)
result = torch.unsqueeze(result, 0)
result = result.permute(0, 2, 1, 3, 4)

print(result.size())

with torch.no_grad():
    output = F.log_softmax(model(result.cuda()), dim=1)
    _, pred = torch.max(output, dim=1)
pred

torch.Size([1, 3, 30, 172, 172])


tensor([2], device='cuda:0')

In [35]:
pred.item()

2

In [55]:
%pwd

'/home/gabir/workspace/repos/ai-system-architecture'

In [44]:
%ls

Dockerfile                        mma/
LICENSE.txt                       movinet_torch.ipynb
MANIFEST.in                       new_Xs.pickle
Makefile                          new_ys.pickle
README.md                         nohup.out
Xs.pickle                         output_video.mp4
Xs_final.pickle                   outputs/
Xs_final_test.pickle              poetry.lock
__MACOSX/                         pyproject.toml
_design/                          requirements.txt
bitbucket-pipelines.yml           runs/
cnn_lstm.ipynb                    scripts/
cuda_11.7.1_515.65.01_linux.run*  setup.cfg
data/                             setup.py
datasets/                         test.jpg
docker_setup.sh                   test1.jpg
entry.py                          test12.jpg
final_dataset/                    tests/
final_dataset_2.zip               weights/
infra/                            ys.pickle
jupyter-venv/                     ys_final.pickle
main.py                           ys_final_test

In [46]:
%cd final_dataset

/home/gabir/workspace/repos/ai-system-architecture/final_dataset


In [47]:
%ls

kicks_01/  kicks_06/  kicks_11/  kicks_16/  kicks_21/  kicks_26/  kicks_31/
kicks_02/  kicks_07/  kicks_12/  kicks_17/  kicks_22/  kicks_27/  kicks_32/
kicks_03/  kicks_08/  kicks_13/  kicks_18/  kicks_23/  kicks_28/  kicks_33/
kicks_04/  kicks_09/  kicks_14/  kicks_19/  kicks_24/  kicks_29/
kicks_05/  kicks_10/  kicks_15/  kicks_20/  kicks_25/  kicks_30/


In [48]:
%cd kicks_01

/home/gabir/workspace/repos/ai-system-architecture/final_dataset/kicks_01


In [49]:
%ls

instances_default.json  kicks_01.mp4  segmented/


In [50]:
%cd segmented

/home/gabir/workspace/repos/ai-system-architecture/final_dataset/kicks_01/segmented


In [51]:
%ls

segment_0.avi   segment_25.avi  segment_41.avi  segment_58.avi  segment_74.avi
segment_1.avi   segment_26.avi  segment_42.avi  segment_59.avi  segment_75.avi
segment_10.avi  segment_27.avi  segment_43.avi  segment_6.avi   segment_76.avi
segment_11.avi  segment_28.avi  segment_44.avi  segment_60.avi  segment_77.avi
segment_12.avi  segment_29.avi  segment_45.avi  segment_61.avi  segment_78.avi
segment_13.avi  segment_3.avi   segment_46.avi  segment_62.avi  segment_79.avi
segment_14.avi  segment_30.avi  segment_47.avi  segment_63.avi  segment_8.avi
segment_15.avi  segment_31.avi  segment_48.avi  segment_64.avi  segment_80.avi
segment_16.avi  segment_32.avi  segment_49.avi  segment_65.avi  segment_81.avi
segment_17.avi  segment_33.avi  segment_5.avi   segment_66.avi  segment_82.avi
segment_18.avi  segment_34.avi  segment_50.avi  segment_67.avi  segment_83.avi
segment_19.avi  segment_35.avi  segment_51.avi  segment_68.avi  segment_84.avi
segment_2.avi   segment_36.avi  segment_52.avi  segme

In [54]:
%cd ..

/home/gabir/workspace/repos/ai-system-architecture
